In [ ]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from datetime import datetime, timedelta
import time

# Specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome('./chromedriver')

# Open the Facebook login page
driver.get("http://www.facebook.com")

# Find the username and password fields, and enter the login details
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

username.clear()
username.send_keys("")
password.clear()
password.send_keys("")

# Click the login button
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Wait for the Facebook homepage to load
driver.implicitly_wait(5)

# Read the list of URLs from a text file
with open("imgUrls.txt", "r") as f:
    urls = f.read().splitlines()

# Create a new directory to store the photos
if not os.path.exists("imgs"):
    os.makedirs("imgs")

# Initialize start time and the index of the last downloaded URL
start_time = datetime.now()
last_saved_index = 0

# Download the photos from each URL
for i in range(len(urls)):
    url = urls[i]
    # Skip URLs that have already been downloaded
    if i < last_saved_index:
        continue
    
    # Navigate to the Facebook page for the current URL
    driver.get(url)

    # Wait for the page to load
    driver.implicitly_wait(5)

    # Set a timeout of 5 seconds for finding the image
    try:
        image = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div/div[1]/div/div[1]/div/div[2]/div/div/div/img")))
        image_url = image.get_attribute("src")
        if image_url:
            # Download the image
            response = requests.get(image_url)

            # Save the image to a file
            filename = f"imgs/{os.path.basename(url)}.jpg"
            with open(filename, "wb") as f:
                f.write(response.content)
                
            # Update the index of the last downloaded URL
            last_saved_index = i

    except:
        # If the image is not found within 10 seconds, skip to the next URL
        print(f"No image found on {url}")
        continue
    
    # Sleep for 5 seconds to prevent being detected as a bot
    time.sleep(5)
    
    # Check if there have been no downloaded images for 0.1 hour
    if i - last_saved_index >= 12:
        # Close the webdriver and restart the program, starting from the last saved index
        driver.quit()
        with open("imgUrls.txt", "w") as f:
            f.write("\n".join(urls[last_saved_index:]))
        os.system("jupyter nbconvert --execute --to notebook --inplace imgDownloader.ipynb")
        os.system("jupyter notebook imgDownloader.ipynb")
        break

#Close the webdriver
driver.quit()
